# Extract Merchant ID and Product ID from link

In [ ]:
import re

def extract_ids(link):

    # Regular expression patterns
    product_id_pattern = r'\/([^\/]+)-p-(\d+)'
    merchant_id_pattern = r'merchantId=(\d+)'

    # Initialize the result dictionary
    result = {
        "product_id_text": None,
        "product_id_number": None,
        "merchant_id": None,
        "product_code": None,
        "product_id": None
    }

    # Extracting product_id
    product_id_match = re.search(product_id_pattern, link)
    if product_id_match:
        result["product_id_text"] = product_id_match.group(1)
        result["product_id_number"] = product_id_match.group(2)

    # Extracting merchant_id
    merchant_id_match = re.search(merchant_id_pattern, link)
    if merchant_id_match:
        result["merchant_id"] = merchant_id_match.group(1)

    if result["product_id_text"] and result["product_id_number"]:
        result["product_code"] = f"{result['product_id_text']}-{result['product_id_number']}"
        result["product_id"] = f"{result['product_id_text']}-p-{result['product_id_number']}"

    return result


# Scrapping Comments

In [ ]:
import requests

def fetch_product_reviews(merchant_id, product_id, max_pages=2):
    comments_list = []

    for rating in range(1, 6):
        for page_number in range(1, max_pages + 1):
            url = f"https://public-mdc.trendyol.com/discovery-web-websfxsocialreviewrating-santral/product-reviews-detailed?sellerId={merchant_id}&contentId={product_id}&page={page_number}&order=DESC&orderBy=Score&rates={rating}&channelId=1"
            try:
                response = requests.get(url)
                response.raise_for_status()  # Ensure we raise an error for bad status codes
                content = response.json()
                comments = content["result"]["productReviews"]["content"]

                for comment in comments:
                    comments_list.append(comment["comment"])

            except requests.exceptions.RequestException as e:
                print(f"Request failed: {e}")
                continue  # Skip the rest and move to the next iteration

    return comments_list


# Scrapping Product Description and Features

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def fetch_product_data(product_code, merchant_id):
    url = f'https://www.trendyol.com/comedones/{product_code}?boutiqueId=61&merchantId={merchant_id}&sav=true'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Check for request errors
    except requests.exceptions.RequestException as e:
        print(f"HTTP Request failed: {e}")
        return None

    text_data = response.text

    try:
        start_index = text_data.index("window.__PRODUCT_DETAIL_APP_INITIAL_STATE") + len("window.__PRODUCT_DETAIL_APP_INITIAL_STATE") + 3
        end_index = text_data.index("};", start_index) + 1
        json_string = text_data[start_index:end_index]
        data = json.loads(json_string)
    except (ValueError, json.JSONDecodeError) as e:
        print(f"Error parsing JSON data: {e}")
        return None

    descriptions = [description["text"] for description in data.get("product", {}).get("descriptions", [])]

    product_dataset = [
        f'{attribute["key"]["name"]} : {attribute["value"]["name"]}'
        for attribute in data.get("product", {}).get("attributes", [])
    ]

    return {
        "descriptions": descriptions,
        "product_dataset": product_dataset,
        "html_content": response.content,
        "text_data": text_data
    }


In [ ]:
import time

def scrapping_pipeline(link):
    # Extract IDs from the provided link
    ids = extract_ids(link)
    merchant_id, product_id, product_code = ids["merchant_id"], ids["product_id_number"], ids["product_id"]
    time.sleep(1)

    if not merchant_id:
        # Fetch product reviews using extracted merchant and product IDs
        comments = fetch_product_reviews("0", product_id)

        time.sleep(1)
        # Fetch product data using product code and merchant ID
        product_details = fetch_product_data(product_code, "0")

        # Concatenate descriptions and product dataset into text strings
        descriptions_text = " ".join(product_details["descriptions"])
        product_dataset_text = " ".join(product_details["product_dataset"])
    else:
        # Fetch product reviews using extracted merchant and product IDs
        comments = fetch_product_reviews(merchant_id, product_id)

        time.sleep(1)
        # Fetch product data using product code and merchant ID
        product_details = fetch_product_data(product_code, merchant_id)

        # Concatenate descriptions and product dataset into text strings
        descriptions_text = " ".join(product_details["descriptions"])
        product_dataset_text = " ".join(product_details["product_dataset"])

    # Return all variables
    return comments, descriptions_text, product_dataset_text


In [ ]:
def question_tamplate(first_product_descriptions_text, first_product_data_text, second_product_descriptions_text, second_product_data_text):

    q_from_tempalate = f"""
    İlk ürünün bilgileri : {first_product_descriptions_text}
    ilk ürünün özellikleri : {first_product_data_text}
    ikinci ürünün bilgileri : {second_product_descriptions_text}
    ikinci ürünün özellikleri : {second_product_data_text}

    Sana iki ürün hakkında bilgi sağladım. Bu bilgileri kullanarak bu iki üründen hangisini alacağı konusunda kararsız kalan bir kullanıcının karar vermek için sorabileceği 10 soruyu bana ver.Cevabında sadece soruları sırala
    """

    return q_from_tempalate

In [ ]:
product_links = [
    ["https://www.trendyol.com/xiaomi/redmi-watch-3-active-akilli-saat-p-744379766?boutiqueId=638145&merchantId=968", "https://www.trendyol.com/seyuwatch/watch-9-pro-akilli-saat-iphone-uyumlu-ve-android-tum-telefonlara-uyumlu-smartwatch-p-706351584?boutiqueId=61&merchantId=803260"],
    ["https://www.trendyol.com/teknoloji-gelsin/bluetooth-kulaklik-kulakici-powerbankli-cift-mikrofonlu-dokunmatik-kablosuz-earbuds-tws-p-218495477?boutiqueId=61&merchantId=117925", "https://www.trendyol.com/deilmi/dots-e6s-bluetooth-kulaklik-universal-hd-ses-ios-android-xiaomi-uyumlu-cift-mikrofon-extra-bass-ae6s-p-125042212?boutiqueId=61&merchantId=248749"],
    ["https://www.trendyol.com/watchofroyal/gunes-gozlugu-kadin-erkek-u v400-cam-ce-belgeli-rose-lorrainew-p-734165777", "https://www.trendyol.com/kinary/unisex-gunes-gozlugu-3-lu-firsat-seti-2028-p-784756196"],
    ["https://www.trendyol.com/philips/speedpro-max-sarjli-dikey-supurge-xc7043-1-elektrikli-25-2v-p-61607778", "https://www.trendyol.com/philips/speedpro-max-aqua-plus-kablosuz-dikey-supurge-aqua-baslik-led-baslik-xc8349-01-p-284597157"],
    ["https://www.trendyol.com/arzum/ar2062-g-airtasty-yagsiz-fitoz-airfryer-4-litre-kapasite-gumus-p-356159741", "https://www.trendyol.com/philips/3000-serisi-airfryer-0-8kg-4-1l-kapasite-siyah-hd9243-90-p-753537583"],
    ["https://www.trendyol.com/apple/iphone-11-64-gb-beyaz-cep-telefonu-aksesuarsiz-kutu-apple-turkiye-garantili-p-65149494", "https://www.trendyol.com/xiaomi/redmi-note-13-pro-12-gb-ram-512-gb-siyah-cep-telefonu-xiaomi-turkiye-garantili-p-797235137"],
    ["https://www.trendyol.com/xiaomi/mi-box-s-4k-android-tv-box-medya-oynatici-2-nesil-p-717630074", "https://www.trendyol.com/xiaomi/mi-tv-stick-4k-ultra-hd-dolby-chromecast-android-tv-media-player-p-301221564"],
    ["https://www.trendyol.com/comedones/dijital-baskul-yag-su-kas-vucut-kitle-endeksi-kilo-olcer-tarti-p-382995037", "https://www.trendyol.com/nivagatore/supscale-dijital-baskul-yag-su-kas-vucut-kitle-endeksi-kilo-olcer-tarti-p-84658219"],
    ["https://www.trendyol.com/kumtel/ko-40-tshdf-siyah-oval-set-ustu-cam-ocak-8mm-p-5608876", "https://www.trendyol.com/luxell/tshdf-siyah-dogalgazli-set-ustu-ankastre-cam-ocak-p-36353123"],
    ["https://www.trendyol.com/nacg/h2o-humidifier-400-ml-ulrasonik-hava-nemlendirici-buhar-makinesi-ve-aroma-difuzoru-h2o-4507-p-765602509?boutiqueId=61&merchantId=803985", "https://www.trendyol.com/heny/300-ml-hava-nemlendirici-buhar-makinasi-rgb-isikli-dekoratif-yanardag-p-805683046"],
    ["https://www.trendyol.com/xiaomi/s10-plus-beyaz-mop-robot-supurge-p-733153362?merchantId=968", "https://www.trendyol.com/roborock/q7-max-beyaz-akilli-robot-supurge-p-316610701?merchantId=106203"]


]


In [ ]:
product_info_map = {}

for i, links in enumerate(product_links):

    link1, link2 = links[0], links[1]

    try:
        first_product_comments, first_product_descriptions_text, first_product_data_text = scrapping_pipeline(link1)
        second_product_comments, second_product_descriptions_text, second_product_data_text = scrapping_pipeline(link2)
    except Exception as e:
        print(f"Error scraping data for product pair {i}: {e}")
        continue

    product_info_map[str(i)] = {
        "first_product_comments": first_product_comments,
        "first_product_descriptions_text": first_product_descriptions_text,
        "first_product_data_text": first_product_data_text,
        "second_product_comments": second_product_comments,
        "second_product_descriptions_text": second_product_descriptions_text,
        "second_product_data_text": second_product_data_text,
        "template" : question_tamplate(first_product_descriptions_text, first_product_data_text, second_product_descriptions_text, second_product_data_text)
    }

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import Markdown
from google.colab import userdata

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')

In [ ]:
!pip install strip-markdown

In [ ]:
import strip_markdown

ai_generatet_questions = {}


for index in product_info_map:
  template = product_info_map[index]["template"]
  response = model.generate_content(template, generation_config=genai.types.GenerationConfig(temperature=0.4))
  plain_text = strip_markdown.strip_markdown(response.text)
  ai_generatet_questions[index] = plain_text
  time.sleep(15)

In [ ]:
!pip install markdown html2text


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=9bf4819dc3c05f6f52e0c9dc0f3c7b708adb4834467801c36b04126dda40958e
  Stored in directory: /root/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built html2text


In [ ]:
import markdown
import html2text

def markdown_to_text(markdown_text):
    html = markdown.markdown(markdown_text)
    text_maker = html2text.HTML2Text()
    text_maker.ignore_links = True
    plain_text = text_maker.handle(html)
    return plain_text


plain_text = markdown_to_text(ai_generatet_questions["3"])
print(plain_text)


İşte iki ürün arasında seçim yapmaya çalışan bir kullanıcının sorabileceği 10
soru:

İki ürünün de toz hazne kapasitesi nedir? Philips Speedpro Max XC7043/01
modelinin su haznesi var mı? Philips Speedpro Max Aqua Plus modelinin şarj
süresi nedir? İki model de halı yıkamada kullanılabilir mi? Hangi model daha
sessiz çalışır? İki model arasındaki fiyat farkı nedir? Hangi model daha hafif
ve kullanımı daha kolay? İki modelin de filtreleri yıkanabilir mi? Hangi model
daha fazla aksesuar ile geliyor? İki modelin de garanti koşulları nelerdir?




In [ ]:
import re

cleaned_ai_generatet_questions = {}

for index in ai_generatet_questions:
    cleaned = re.findall(r'[^\n]+[?]', ai_generatet_questions[index])
    cleaned_ai_generatet_questions[index] = cleaned

In [ ]:
cleaned_ai_generatet_questions

In [ ]:
product_info_map

In [ ]:
!pip install faiss-cpu
!pip install langchain
!pip install sentence-transformers
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "intfloat/multilingual-e5-large"
model_kwargs = {"device": "cpu"} #cpu or cuda

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
def generate_vector_store(dataset, embeddings):
  first_comment_vectorstore = FAISS.from_texts(dataset, embeddings)
  return first_comment_vectorstore

In [ ]:
embedding_map = {}

for index in product_info_map:
    try:
        first_product_comment_data = product_info_map[index]["first_product_comments"]
        second_product_comment_data = product_info_map[index]["second_product_comments"]
        first_vector_store = generate_vector_store(first_product_comment_data, embeddings)
        second_vector_store = generate_vector_store(second_product_comment_data, embeddings)
        embedding_map[index] = {"first_product_vs" : first_vector_store, "second_product_vs" : second_vector_store}
    except:
        print("An error occured")
        embedding_map[index] = {"first_product_vs" : "", "second_product_vs" : ""}




0 - -- -  ["Arkadaşlar GARANTİ'ye dikkat edin.Ürünü resmi distribütör garantili diye satıyorlar fakat gelen ürünün görüntüsünü ekliyorum.Xiaomi yetkili servislerinden iki tanesi ile görüştüm seri numarası ile baktılar bize ait değil dedi ikiside.Ürünün üzerindeki firmayı aradım onlarda seri numarası ile baktılar onlarda bize ait değil dediler.Trendyol ile kaç kere görüştüm şuan ürünü Trendyola geri gönderdim analize bakalım sonuç ne olacak", 'Ürün iade seçeneği yok beklediğimiz çok büyük geldi', 'Parasına değicek bi ürün değil ya bence', 'çoğu bildirimi acmadigim halde 5 gün dayanmıyor şarjı.Ayrica %25 şarj a gelince saat çok hızlı şekilde pili bitiyor. Tavsiye etmiyorum.', 'Aynı böyle geldi kargo. Bunun içinde saat var kırılabilir. Biraz dikkat etmez mi insan', 'Ben aldiktan sonra indirime girdi her zamanki gibi', 'gerçekten çalıştığınız kargo ile anlaşmaya son verip yeni bir kargo firması ile anlaşmalısınız , evimize dahi getirmediler gidip şubeden aldık ve bu şekilde paket hasar gör

In [ ]:
product_info_map["8"]["first_product_comments"] ,product_info_map["8"]["second_product_comments"]

(['Ürün güzel ama NE YAZİKKİ İSTANBUL ANADOLU YAKASI SERVİSİ ÇOK İLGİSİZ 3 GÜNDÜR BUGÜN GELİYORUZ BUGÜN GELİYORUZ DİYEREK OYALIYORLAR…!!! Keşke almasaydım dedim',
  'idare eder',
  'çakmaklık çalışmıyor',
  'Kargo çok zayıf satıcı firma ilgisiz umursamıyor',
  'berbat bi ürün arkadaşlar almayın',
  'kutu hasarlı geldi daha açılmadı bakalım????',
  'servis kuruluma gelmiyor , kendimiz masraf ederek kurduk.',
  'acelesi olduğu için iade etmedim ama burdan alacağınıza mağazadan almanız daha mantıklı çakmakları doğru düzgün çalışmıyor bile paranızı vermeye değmez bir Yıldız dahi hak etmiyor maalesef',
  'urun geldi deniyecegiz umariz bekletilmemize deger ama şunun altini cizerek belirtmekte isterim urun 12 gün sonra geldi, sipariş verirken ona gore alın malum bütün anadoluyu geziyor bilginiz olsun kargolarla irtibatiniza dikkat edin lütfen ve umarım ki iade etmeye gerek kalacak birseyle karsilazmayizki bi 12 gün daha beklemeyelim !!!',
  'Kumtel çağrı merkezini aradım 3 iş günü içerisinde 

In [ ]:
# first_product_comment_data = product_info_map["5"]

In [ ]:
def create_template(first_product_descriptions, first_product_features, first_product_comments, second_product_descriptions, second_product_features, second_product_comments, user_question):
    template = f"Bir yapay zeka alışveriş asistanı olarak rol yapmanı istiyorum. İki farklı ürün hakkında bana bilgi verdiğimde, bir alışveriş asistanının yapacağı gibi sorularıma cevap vereceksin.\
    Ana görevin, önerilerde bulunmak, iki ürün arasında karşılaştırmalar yapmak ve kalite, fiyat, amaçlanan kullanıma uygunluk ve mevcutsa yorumlar gibi çeşitli faktörleri göz önünde bulundurarak hangi ürünün daha iyi bir seçim olabileceği konusunda tavsiyelerde bulunmak olacaktır. \
    Verdiğim ürün bilgileri dışında herhangi bir ek bilgi vermemelisin.\
    İlk ürünün açıklaması : {first_product_descriptions}, İlk ürünün özellikleri : {first_product_features}, İlk ürünün yorumları : {first_product_comments}\
    ikinci ürünün açıklaması : {second_product_descriptions}, ikinci ürünün özellikleri : {second_product_features}, ikinci ürünün yorumları : {second_product_comments} \
    Kullanıcı sorusu : {user_question}"

    return template

In [ ]:
create_template("first_product_descriptions", "first_product_features", "first_product_comments", "second_product_descriptions", "second_product_features", "second_product_comments", "user_question")


'Bir yapay zeka alışveriş asistanı olarak rol yapmanı istiyorum. İki farklı ürün hakkında bana bilgi verdiğimde, bir alışveriş asistanının yapacağı gibi sorularıma cevap vereceksin.    Ana görevin, önerilerde bulunmak, iki ürün arasında karşılaştırmalar yapmak ve kalite, fiyat, amaçlanan kullanıma uygunluk ve mevcutsa yorumlar gibi çeşitli faktörleri göz önünde bulundurarak hangi ürünün daha iyi bir seçim olabileceği konusunda tavsiyelerde bulunmak olacaktır.     Verdiğim ürün bilgileri dışında herhangi bir ek bilgi vermemelisin.    İlk ürünün açıklaması : first_product_descriptions, İlk ürünün özellikleri : first_product_features, İlk ürünün yorumları : first_product_comments    ikinci ürünün açıklaması : second_product_descriptions, ikinci ürünün özellikleri : second_product_features, ikinci ürünün yorumları : second_product_comments     Kullanıcı sorusu : user_question'

In [ ]:
def faiss_document_to_array(documents):
    document_array = []
    for document in documents:
        document_array.append(document.page_content)
    return document_array

In [ ]:
generated_templates = {}

for index in product_info_map:
    generated_templates[index] = {}
    try:
        questions = cleaned_ai_generatet_questions[index]
        for question_index in range(len(questions)):
            user_question = questions[question_index]
            first_product_descriptions = product_info_map[index]["first_product_descriptions_text"]
            first_product_features = product_info_map[index]["first_product_data_text"]
            first_product_comments = embedding_map[index]["first_product_vs"].similarity_search(user_question, k=15)
            first_product_comments_array = faiss_document_to_array(first_product_comments)
            first_product_comments_formated = ", ".join(first_product_comments_array)


            second_product_descriptions = product_info_map[index]["second_product_descriptions_text"]
            second_product_features = product_info_map[index]["second_product_data_text"]
            second_product_comments = embedding_map[index]["second_product_vs"].similarity_search(user_question, k=15)
            second_product_comments_array = faiss_document_to_array(second_product_comments)
            second_product_comments_formated = ", ".join(second_product_comments_array)

            template = create_template(first_product_descriptions, first_product_features, first_product_comments_formated, second_product_descriptions, second_product_features, second_product_comments_formated, user_question)
            generated_templates[index][question_index] = template
    except:
        print("Error for index : " + index)

Error for index : 8


In [ ]:
generated_templates["2"][5]

'Bir yapay zeka alışveriş asistanı olarak rol yapmanı istiyorum. İki farklı ürün hakkında bana bilgi verdiğimde, bir alışveriş asistanının yapacağı gibi sorularıma cevap vereceksin.    Ana görevin, önerilerde bulunmak, iki ürün arasında karşılaştırmalar yapmak ve kalite, fiyat, amaçlanan kullanıma uygunluk ve mevcutsa yorumlar gibi çeşitli faktörleri göz önünde bulundurarak hangi ürünün daha iyi bir seçim olabileceği konusunda tavsiyelerde bulunmak olacaktır.     Verdiğim ürün bilgileri dışında herhangi bir ek bilgi vermemelisin.    İlk ürünün açıklaması : Bu ürün\xa0[page="merchant_info"]WATCHOFROYAL[/page]\xa0tarafından gönderilecektir. Kampanya fiyatından satılmak üzere 100 adetten fazla stok sunulmuştur. İncelemiş olduğunuz ürünün satış fiyatını satıcı belirlemektedir. Bir ürün, birden fazla satıcı tarafından satılabilir. Birden fazla satıcı tarafından satışa sunulan ürünlerin satıcıları ürün için belirledikleri fiyata, satıcı puanlarına, teslimat statülerine, ürünlerdeki promosyon

In [ ]:
generated_templates

In [ ]:
len(generated_templates["0"])

10

In [ ]:
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import Markdown

In [ ]:
from google.colab import userdata

ai_model = 'gemini-1.5-pro-latest'
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel(ai_model)

In [ ]:
!pip install strip-markdown

In [ ]:
import strip_markdown
import time

ai_generated_answers = {}


for index in generated_templates:
    ai_generated_answers[index] = {}
    for question_index in generated_templates[index]:
        question = generated_templates[index][question_index]
        response = model.generate_content(question, generation_config=genai.types.GenerationConfig(temperature=0.3))
        plain_text = strip_markdown.strip_markdown(response.text)

        ai_generated_answers[index][question_index] = {}

        ai_generated_answers[index][question_index]["question"] = question
        ai_generated_answers[index][question_index]["response"] = plain_text
        ai_generated_answers[index][question_index]["ai_model"] = ai_model

        time.sleep(15)

In [2]:
ai_generated_answers

{'0': {0: {'question': 'Bir yapay zeka alışveriş asistanı olarak rol yapmanı istiyorum. İki farklı ürün hakkında bana bilgi verdiğimde, bir alışveriş asistanının yapacağı gibi sorularıma cevap vereceksin.    Ana görevin, önerilerde bulunmak, iki ürün arasında karşılaştırmalar yapmak ve kalite, fiyat, amaçlanan kullanıma uygunluk ve mevcutsa yorumlar gibi çeşitli faktörleri göz önünde bulundurarak hangi ürünün daha iyi bir seçim olabileceği konusunda tavsiyelerde bulunmak olacaktır.     Verdiğim ürün bilgileri dışında herhangi bir ek bilgi vermemelisin.    İlk ürünün açıklaması : Bu ürün\xa0[page="merchant_info"]Trendyol[/page]\xa0tarafından gönderilecektir. Kampanya fiyatından satılmak üzere 100 adetten fazla stok sunulmuştur. Ürüne ait garanti belgesi ile tanıtma ve kullanım kılavuzları; kağıt ortamında paylaşılabileceği gibi elektronik ortamda erişim sağlayabileceğiniz, link ve karekod biçiminde de sunulabilecektir. Ürün teslimi sonrası ambalajın, paketin, kutunun dikkatle açılması t

In [ ]:
ai_generated_answers["0"][0]

{'question': 'Bir yapay zeka alışveriş asistanı olarak rol yapmanı istiyorum. İki farklı ürün hakkında bana bilgi verdiğimde, bir alışveriş asistanının yapacağı gibi sorularıma cevap vereceksin.    Ana görevin, önerilerde bulunmak, iki ürün arasında karşılaştırmalar yapmak ve kalite, fiyat, amaçlanan kullanıma uygunluk ve mevcutsa yorumlar gibi çeşitli faktörleri göz önünde bulundurarak hangi ürünün daha iyi bir seçim olabileceği konusunda tavsiyelerde bulunmak olacaktır.     Verdiğim ürün bilgileri dışında herhangi bir ek bilgi vermemelisin.    İlk ürünün açıklaması : Bu ürün\xa0[page="merchant_info"]Trendyol[/page]\xa0tarafından gönderilecektir. Kampanya fiyatından satılmak üzere 100 adetten fazla stok sunulmuştur. Ürüne ait garanti belgesi ile tanıtma ve kullanım kılavuzları; kağıt ortamında paylaşılabileceği gibi elektronik ortamda erişim sağlayabileceğiniz, link ve karekod biçiminde de sunulabilecektir. Ürün teslimi sonrası ambalajın, paketin, kutunun dikkatle açılması tavsiye edi

In [ ]:
train = []
test = []
for product_index in  ai_generated_answers:
    for question_index in ai_generated_answers[product_index]:
        if product_index == "10":
            test.append([ai_generated_answers[product_index][question_index]["question"], ai_generated_answers[product_index][question_index]["response"]])
        else:
            train.append([ai_generated_answers[product_index][question_index]["question"], ai_generated_answers[product_index][question_index]["response"]])

In [3]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset
import pandas as pd

dataset = Dataset.from_pandas(pd.DataFrame(train))
pd.DataFrame(train).to_csv("train_dataset.csv")

In [ ]:
dataset[0]

In [ ]:
from datasets import load_dataset

alpaca_prompt = """
{}

{}

"""

# Function to format the prompts
def formatting_prompts_func(examples):
    instructions = examples["0"]  # Use the 'quote' column as the instruction
    inputs = examples["1"]       # Use the 'author' column as the input
       # Use the 'tags' column as the output
    texts = []
    for instruction, input in zip(instructions, inputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Load the custom CSV dataset
dataset = load_dataset('csv', data_files='/content/train_dataset.csv', split='train')

# Apply the formatting function to your custom dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# alpaca_prompt = Copied from above
def predict_tags(model, quote, author):

    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            quote, # instruction
            author, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    predicted_value = tokenizer.batch_decode(outputs)
    return predicted_value

In [ ]:
for data in test_dataset:
    print(predict_tags(model, data["quote"], data["author"]))
    print("trues : " + data["tags"])